In [16]:

#Dr. Ma, I converted this to a jupyter notebook within pycharm and am working on this in jupyter notebook as I prefer the UI much more than pycharm. Is this okay? 
import os
import numpy as np
import scipy.io as sio
from self_py_fun.HW10Fun import *
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


# In HW7, you have the chance to visualize a truncated EEG dataset stratified by
# target and non-target stimulus type.
#
# The fundamental problem of P300 ERP-BCI speller system is to perform a binary classification.
#
# In HW10, you are asked to implement the binary classification using various methods,
# and evaluate the model performance with a testing dataset.
#
# You will use K114_001_BCI_TRN_Truncated_Data_0.5_6.mat as a training set, and
# K114_001_BCI_FRT_Truncated_Data_0.5_6.mat as a testing set.
#
# Notice that here, we do not split training/testing within K114_001_BCI_TRN_Truncated_Data_0.5_6.mat
# because each row is not entirely independent of each other due to the special structure of the dataset.

# Global constants:
np.random.seed(100)
bp_low = 0.5
bp_upp = 6
electrode_num = 16
# Change the following directory to your own one.
parent_dir = os.getcwd()
parent_data_dir = '{}/data'.format(parent_dir)
time_index = np.linspace(0, 800, 25)
electrode_name_ls = ['F3', 'Fz', 'F4', 'T7', 'C3', 'Cz', 'C4', 'T8', 'CP3', 'CP4', 'P3', 'Pz', 'P4', 'PO7', 'PO8', 'Oz']
subject_name = 'K114'
# create a new folder called K114
subject_dir = '{}/{}'.format(parent_dir, subject_name)
if not os.path.exists(subject_dir):
    os.mkdir(subject_dir)

char_trn = 'THE0QUICK0BROWN0FOX' #19
char_trn_size = len(char_trn)
print(char_trn_size) #also 19
# Step 1: Import dataset
# Step 1.1: TRN dataset
trn_data_name = '{}_001_BCI_TRN_Truncated_Data_{}_{}'.format(subject_name, bp_low, bp_upp)
trn_data_dir = '{}/{}.mat'.format(parent_data_dir, trn_data_name)
eeg_trn_obj = sio.loadmat(trn_data_dir)

# eeg_trn_obj is a dictionary!
print(eeg_trn_obj.keys())
eeg_trn_signal = eeg_trn_obj['Signal']
print(eeg_trn_signal.shape) # 3420, 400
eeg_trn_type = eeg_trn_obj['Type']
print(eeg_trn_type.shape) # 3420, 1
eeg_trn_type = np.squeeze(eeg_trn_type, axis=1)

# Step 1.2: FRT dataset
# The following code should be completed by students themselves.
# you should be able to obtain relevant data files named
# eeg_frt_signal and eeg_frt_type
# Write your own code below:

# Cell: load FRT dataset and prepare related variables
from pathlib import Path

parent_data_dir = Path(parent_data_dir)

subject_name = globals().get("subject_name", "K114")
bp_low = globals().get("bp_low", 0.5)
bp_upp = globals().get("bp_upp", 6)

frt_data_name = f"{subject_name}_001_BCI_FRT_Truncated_Data_{bp_low}_{bp_upp}"
frt_path = parent_data_dir / f"{frt_data_name}.mat"

# Existence check
if not frt_path.exists():
    raise FileNotFoundError(f"FRT .mat not found at: {frt_path}")

# Load .mat
eeg_frt_obj = sio.loadmat(str(frt_path))
print("FRT keys:", list(eeg_frt_obj.keys()))

# Extract Signal and Type with defensive checks
if 'Signal' not in eeg_frt_obj or 'Type' not in eeg_frt_obj:
    raise KeyError("Expected keys 'Signal' and 'Type' not found in the FRT .mat file.")

eeg_frt_signal = eeg_frt_obj['Signal']
eeg_frt_type = eeg_frt_obj['Type']

# Ensure shapes and dtypes are convenient
print("raw Signal shape:", eeg_frt_signal.shape, "raw Type shape:", eeg_frt_type.shape)

# If Type has shape (n,1) or (n, ), squeeze to (n,)
eeg_frt_type = np.squeeze(eeg_frt_type)
# Convert to integer labels if appropriate
try:
    eeg_frt_type = eeg_frt_type.astype(int)
except Exception:
    pass

# If Signal is 3D (n, channels, times), flatten to (n, channels*times) for classic classifiers
if eeg_frt_signal.ndim == 3:
    n_frt = eeg_frt_signal.shape[0]
    eeg_frt_signal = eeg_frt_signal.reshape(n_frt, -1)

print("processed Signal shape:", eeg_frt_signal.shape, "processed Type shape:", eeg_frt_type.shape)

# Code and Text fields (if present)
eeg_frt_code = eeg_frt_obj.get('Code', None)
eeg_frt_text = eeg_frt_obj.get('Text', None)

# Derive char_frt using helper if available, otherwise fallback to simple conversion
char_frt = None
if 'convert_raw_char_to_alphanumeric_stype' in globals():
    try:
        char_frt = convert_raw_char_to_alphanumeric_stype(eeg_frt_text)
    except Exception:
        char_frt = None

if char_frt is None:
    # Fallback: if Text is an array of characters or a string, try to convert
    if eeg_frt_text is None:
        raise ValueError("No 'Text' field found in FRT .mat and helper conversion failed.")
    # handle common MATLAB string formats
    if isinstance(eeg_frt_text, np.ndarray):
        # try to join characters or decode bytes
        try:
            # if it's an array of shape (1,) with a string
            if eeg_frt_text.size == 1:
                raw = eeg_frt_text.flatten()[0]
                char_frt = list(str(raw))
            else:
                # flatten and join rows
                char_frt = [str(x).strip() for x in eeg_frt_text.flatten()]
        except Exception:
            char_frt = list(str(eeg_frt_text))
    else:
        char_frt = list(str(eeg_frt_text))

# Finalize char_frt and sizes
char_frt = list(char_frt)
char_frt_size = len(char_frt)
print("char_frt_size:", char_frt_size)

# Compute sequence sizes using the same logic as training code
# (adjust divisor if your dataset uses a different grouping than 12)
trn_seq_size = globals().get("trn_seq_size", None)
if trn_seq_size is None and 'eeg_trn_signal' in globals() and 'char_trn_size' in globals():
    trn_seq_size = int(eeg_trn_signal.shape[0] / char_trn_size / 12)

frt_seq_size = int(eeg_frt_signal.shape[0] / char_frt_size / 12)
print("trn_seq_size:", trn_seq_size, "frt_seq_size:", frt_seq_size)

# Print a small sanity check of the first few labels and codes
print("first 10 eeg_frt_type labels:", eeg_frt_type[:10])
if eeg_frt_code is not None:
    print("first 10 eeg_frt_code rows:", eeg_frt_code[:10])



# You have completed the exploratory data analysis in HW7 and HW8.
# The dataset has been carefully reviewed by Dr. Jane E. Huggins,
# so we do not need to worry about missing, outliers, errors of the dataset.

# Step 2: Fit classification models
# You will try the following methods:
# Logistic Regression,
# Linear Discriminant Analysis,
# Support Vector Machine (sometimes called support vector classification)
# You do not need to modify the parameters of each classifier
# except for LogisticRegression: set max_iter=1000
# Write your own code below:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

# Ensure the required data variables exist
X_trn = eeg_trn_signal
y_trn = np.squeeze(eeg_trn_type).astype(int)

X_frt = eeg_frt_signal
y_frt = np.squeeze(eeg_frt_type).astype(int)

# Flatten if needed (AI says this is where my error was before)
if X_trn.ndim == 3:
    X_trn = X_trn.reshape(X_trn.shape[0], -1)
if X_frt.ndim == 3:
    X_frt = X_frt.reshape(X_frt.shape[0], -1)

# Scale features
scaler = StandardScaler()
X_trn_scaled = scaler.fit_transform(X_trn)
X_frt_scaled = scaler.transform(X_frt)

# Ensure model-output arrays are column vectors (n_events, 1) for streamline_predict
import numpy as np

logistic_y_trn = np.asarray(logistic_y_trn).ravel()[:, np.newaxis]
lda_y_trn      = np.asarray(lda_y_trn).ravel()[:, np.newaxis]
svm_y_trn      = np.asarray(svm_y_trn).ravel()[:, np.newaxis]

logistic_y_frt = np.asarray(logistic_y_frt).ravel()[:, np.newaxis]
lda_y_frt      = np.asarray(lda_y_frt).ravel()[:, np.newaxis]
svm_y_frt      = np.asarray(svm_y_frt).ravel()[:, np.newaxis]

print("Shapes:", logistic_y_trn.shape, lda_y_trn.shape, svm_y_trn.shape,
      logistic_y_frt.shape, lda_y_frt.shape, svm_y_frt.shape)

# Logistic Regression
log_clf = LogisticRegression(max_iter=1000, random_state=100)
log_clf.fit(X_trn_scaled, y_trn)
logistic_y_trn = log_clf.predict_proba(X_trn_scaled)[:, 1]
logistic_y_frt = log_clf.predict_proba(X_frt_scaled)[:, 1]

# LDA
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_trn_scaled, y_trn)
if hasattr(lda_clf, "predict_proba"):
    lda_y_trn = lda_clf.predict_proba(X_trn_scaled)[:, 1]
    lda_y_frt = lda_clf.predict_proba(X_frt_scaled)[:, 1]
else:
    lda_y_trn = lda_clf.decision_function(X_trn_scaled)
    lda_y_frt = lda_clf.decision_function(X_frt_scaled)

svm_clf = SVC(kernel='rbf', probability=True, random_state=100)
svm_clf.fit(X_trn_scaled, y_trn)
svm_y_trn = svm_clf.predict_proba(X_trn_scaled)[:, 1]
svm_y_frt = svm_clf.predict_proba(X_frt_scaled)[:, 1]

trained_models = {
    "logistic": log_clf,
    "lda": lda_clf,
    "svm": svm_clf,
    "scaler": scaler
}

#Printing out step 2 results because this is all in 1 code chunk and the output is LONG
print("Created model-output arrays with shapes:")
print("logistic_y_trn", logistic_y_trn.shape, "logistic_y_frt", logistic_y_frt.shape)
print("lda_y_trn", np.shape(lda_y_trn), "lda_y_frt", np.shape(lda_y_frt))
print("svm_y_trn", svm_y_trn.shape, "svm_y_frt", svm_y_frt.shape)


# Step 3: Evaluate model performance on both TRN and FRT files
# Step 3.1: Prediction accuracy on TRN files
# We will compute the probability of each stimulus with .predict_proba()
# before we convert the stimulus-level probability to character-level probability.
# You are asked to generate stimulus-level probability for each method on TRN files,
# denoted as logistic_y_trn, lda_y_trn, and svm_y_trn.
# Write your own code below:
# Step 3.1 — stimulus-level evaluation on TRN
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

# Ensure data variables exist
assert 'eeg_trn_signal' in globals() and 'eeg_trn_type' in globals(), "Load TRN data first."

# Prepare binary labels for metrics: target==1 -> True, non-target otherwise
y_trn_raw = np.squeeze(eeg_trn_type)
y_trn_bin = (y_trn_raw == 1).astype(int)

# If any model-output arrays are missing, try to compute them using trained models in trained_models
if 'logistic_y_trn' not in globals() or 'lda_y_trn' not in globals() or 'svm_y_trn' not in globals():
    print("Some model-output arrays missing; attempting to compute from trained_models if available.")
    # require trained_models and scaler
    if 'trained_models' in globals() and 'scaler' in globals():
        scaler = trained_models.get("scaler", globals().get("scaler"))
        X_trn = eeg_trn_signal
        if X_trn.ndim == 3:
            X_trn = X_trn.reshape(X_trn.shape[0], -1)
        X_trn_scaled = scaler.transform(X_trn) if hasattr(scaler, "transform") else X_trn
        # logistic
        if 'logistic_y_trn' not in globals() and 'logistic' in trained_models:
            clf = trained_models['logistic']
            logistic_y_trn = clf.predict_proba(X_trn_scaled)[:, 1]
        # lda
        if 'lda_y_trn' not in globals() and 'lda' in trained_models:
            clf = trained_models['lda']
            if hasattr(clf, "predict_proba"):
                lda_y_trn = clf.predict_proba(X_trn_scaled)[:, 1]
            else:
                lda_y_trn = clf.decision_function(X_trn_scaled)
        # svm
        if 'svm_y_trn' not in globals() and 'svm' in trained_models:
            clf = trained_models['svm']
            if hasattr(clf, "predict_proba"):
                svm_y_trn = clf.predict_proba(X_trn_scaled)[:, 1]
            else:
                svm_y_trn = clf.decision_function(X_trn_scaled)
    else:
        raise RuntimeError("Model outputs missing and no trained_models/scaler available to compute them.")

# Collect methods and arrays
methods = {
    "Logistic": globals().get("logistic_y_trn"),
    "LDA"     : globals().get("lda_y_trn"),
    "SVM"     : globals().get("svm_y_trn")
}

# Evaluate each method at stimulus level
for name, scores in methods.items():
    if scores is None:
        print(f"{name}: no scores found, skipping.")
        continue
    # Ensure 1D numpy array
    scores = np.asarray(scores).ravel()
    if scores.shape[0] != y_trn_bin.shape[0]:
        print(f"{name}: length mismatch (scores {scores.shape[0]} vs labels {y_trn_bin.shape[0]}). Skipping.")
        continue

    # AUC (requires both classes present)
    try:
        auc = roc_auc_score(y_trn_bin, scores)
    except Exception as e:
        auc = f"NA ({e})"

    # Accuracy at threshold 0.5 (convert scores to predicted labels)
    y_pred = (scores >= 0.5).astype(int)
    acc = accuracy_score(y_trn_bin, y_pred)
    cm = confusion_matrix(y_trn_bin, y_pred)

    print(f"\n{name} (stimulus-level):")
    print("  n_events:", scores.shape[0])
    print("  AUC:", auc)
    print("  Accuracy @0.5:", acc)
    print("  Confusion matrix (rows=true [0,1], cols=pred [0,1]):")
    print(cm)

# Step 3.2: Prediction accuracy on FRT files
# Similarly, you are asked to generate stimulus-level probability for each method on FRT files,
# denoted as logistic_y_frt, lda_y_frt, and svm_y_frt.
# Write your own code below:


# Step 3.2: ensure stimulus-level probabilities and FRT metadata exist
import numpy as np

# --- basic checks ---
assert 'eeg_frt_signal' in globals() and 'eeg_frt_obj' in globals(), "Load FRT .mat first."

# expose code field and text/char info used by Step 4
eeg_frt_code = eeg_frt_obj['Code']
# prefer helper conversion if available
if 'convert_raw_char_to_alphanumeric_stype' in globals():
    try:
        char_frt = convert_raw_char_to_alphanumeric_stype(eeg_frt_obj['Text'])
    except Exception:
        char_frt = list(str(eeg_frt_obj.get('Text', '')))
else:
    # fallback: try to extract readable text from the Text field
    raw_text = eeg_frt_obj.get('Text', '')
    if isinstance(raw_text, np.ndarray) and raw_text.size == 1:
        char_frt = list(str(raw_text.flatten()[0]))
    elif isinstance(raw_text, np.ndarray):
        char_frt = [str(x).strip() for x in raw_text.flatten()]
    else:
        char_frt = list(str(raw_text))

char_frt = list(char_frt)
char_frt_size = len(char_frt)
frt_seq_size = int(eeg_frt_signal.shape[0] / char_frt_size / 12)

# --- compute/confirm model-output arrays for FRT ---
# If already present, keep them; otherwise compute from trained models or individual classifiers
def _ensure_scores(name, clf_key=None, clf_var=None):
    if name in globals():
        arr = np.asarray(globals()[name]).ravel()
        return arr
    # try trained_models dict first
    if 'trained_models' in globals() and clf_key in trained_models:
        clf = trained_models[clf_key]
    elif clf_var is not None and clf_var in globals():
        clf = globals()[clf_var]
    else:
        return None
    X = eeg_frt_signal
    if X.ndim == 3:
        X = X.reshape(X.shape[0], -1)
    scaler = trained_models.get('scaler') if 'trained_models' in globals() else globals().get('scaler', None)
    if scaler is not None and hasattr(scaler, "transform"):
        Xs = scaler.transform(X)
    else:
        Xs = X
    if hasattr(clf, "predict_proba"):
        return clf.predict_proba(Xs)[:, 1].ravel()
    elif hasattr(clf, "decision_function"):
        return clf.decision_function(Xs).ravel()
    else:
        return None

logistic_y_frt = _ensure_scores('logistic_y_frt', clf_key='logistic', clf_var='log_clf')
lda_y_frt      = _ensure_scores('lda_y_frt',      clf_key='lda',      clf_var='lda_clf')
svm_y_frt      = _ensure_scores('svm_y_frt',      clf_key='svm',      clf_var='svm_clf')

# final sanity: raise clear error if any required score is still missing
missing_scores = [n for n in ['logistic_y_frt','lda_y_frt','svm_y_frt'] if globals().get(n) is None]
if missing_scores:
    raise RuntimeError(f"Missing FRT model-output arrays: {missing_scores}. Re-run Step 2 to train models or ensure trained_models/scaler exist.")

# coerce to 1D numpy arrays
# after computing logistic_y_frt, lda_y_frt, svm_y_frt
import numpy as np
logistic_y_frt = np.asarray(logistic_y_frt).ravel()[:, np.newaxis]
lda_y_frt      = np.asarray(lda_y_frt).ravel()[:, np.newaxis]
svm_y_frt      = np.asarray(svm_y_frt).ravel()[:, np.newaxis]


# quick confirmation print (safe, minimal)
print("Prepared FRT arrays:", logistic_y_frt.shape, lda_y_frt.shape, svm_y_frt.shape)
print("char_frt_size:", char_frt_size, "frt_seq_size:", frt_seq_size)



# Step 4: Convert binary classification probability to character-level accuracy
# This involves advanced data manipulation, so you do not need to write any new code.
# Please run the following code to view the final results.
eeg_trn_code = eeg_trn_obj['Code']
eeg_frt_code = eeg_frt_obj['Code']
char_frt = convert_raw_char_to_alphanumeric_stype(eeg_frt_obj['Text'])
# raw format is different from the current 6x6 layout characters.
char_frt_size = len(char_frt)
frt_seq_size = int(eeg_frt_signal.shape[0]/char_frt_size/12)

# --- Ensure model-output arrays are column vectors right before evaluation ---
import numpy as np

for name in ['logistic_y_trn','lda_y_trn','svm_y_trn','logistic_y_frt','lda_y_frt','svm_y_frt']:
    if name not in globals():
        raise RuntimeError(f"{name} not found in globals() — run Step 2/3 first.")
    arr = np.asarray(globals()[name]).ravel()
    globals()[name] = arr[:, np.newaxis]

# Confirm shapes
print("Final shapes before streamline_predict:",
      logistic_y_trn.shape, lda_y_trn.shape, svm_y_trn.shape,
      logistic_y_frt.shape, lda_y_frt.shape, svm_y_frt.shape)

# Logistic regression
print('Logistic Regression on TRN:')
logistic_letter_mat_trn, logistic_letter_prob_mat_trn = streamline_predict(
    logistic_y_trn, eeg_trn_type, eeg_trn_code, char_trn_size, trn_seq_size,
    stimulus_group_set, eeg_rcp_array
)
print(logistic_letter_mat_trn)
print(list(char_trn)) # This is the true spelling characters for training set!
logistic_trn_accuracy = np.mean(logistic_letter_mat_trn == np.array(list(char_trn))[:, np.newaxis], axis=0)

print('Logistic Regression on FRT:')
logistic_letter_mat_frt, logistic_letter_prob_mat_frt = streamline_predict(
    logistic_y_frt, eeg_frt_type, eeg_frt_code, char_frt_size, frt_seq_size,
    stimulus_group_set, eeg_rcp_array
)
print(logistic_letter_mat_frt)
print(list(char_frt)) # This is the true spelling characters for testing set!
logistic_frt_accuracy = np.mean(logistic_letter_mat_frt == np.array(list(char_frt))[:, np.newaxis], axis=0)

# LDA:
print('LDA on TRN:')
lda_letter_mat_trn, lda_letter_prob_mat_trn = streamline_predict(
    lda_y_trn, eeg_trn_type, eeg_trn_code, char_trn_size, trn_seq_size,
    stimulus_group_set, eeg_rcp_array
)
print(lda_letter_mat_trn)
print(list(char_trn)) # This is the true spelling characters for training set!
lda_trn_accuracy = np.mean(lda_letter_mat_trn == np.array(list(char_trn))[:, np.newaxis], axis=0)

print('LDA on FRT:')
lda_letter_mat_frt, lda_letter_prob_mat_frt = streamline_predict(
    lda_y_frt, eeg_frt_type, eeg_frt_code, char_frt_size, frt_seq_size,
    stimulus_group_set, eeg_rcp_array
)
print(lda_letter_mat_frt)
print(list(char_frt)) # This is the true spelling characters for testing set!
lda_frt_accuracy = np.mean(lda_letter_mat_frt == np.array(list(char_frt))[:, np.newaxis], axis=0)

# SVM:
print('Support Vector Machine on TRN:')
svm_letter_mat_trn, svm_letter_prob_mat_trn = streamline_predict(
    svm_y_trn, eeg_trn_type, eeg_trn_code, char_trn_size, trn_seq_size,
    stimulus_group_set, eeg_rcp_array
)
print(svm_letter_mat_trn)
print(list(char_trn)) # This is the true spelling characters for training set!
svm_trn_accuracy = np.mean(svm_letter_mat_trn == np.array(list(char_trn))[:, np.newaxis], axis=0)

print('Support Vector Machine on FRT:')
svm_letter_mat_frt, svm_letter_prob_mat_frt = streamline_predict(
    svm_y_frt, eeg_frt_type, eeg_frt_code, char_frt_size, frt_seq_size,
    stimulus_group_set, eeg_rcp_array
)
print(svm_letter_mat_frt)
print(list(char_frt)) # This is the true spelling characters for training set!
svm_frt_accuracy = np.mean(svm_letter_mat_frt == np.array(list(char_frt))[:, np.newaxis], axis=0)


print("Logistic trn accuracy:",logistic_trn_accuracy) # I added labels so I know what I'm looking at if that's okay
print("lda trn accuracy:",lda_trn_accuracy)
print("svm trn accuracy:",svm_trn_accuracy)

print("log frt acc:",logistic_frt_accuracy)
print("lds frt acc:",lda_frt_accuracy)
print("svm frt acc:",svm_frt_accuracy)


# Remember to answer two questions below:

# What do rows 122, 131, 141, 150, 160, and 169 do? Briefly answer the question below:
# In case that your row IDs are messed up when you start to fill in the blank,
# I attach the lines of code for your reference.
# logistic_trn_accuracy = np.mean(logistic_letter_mat_trn == np.array(list(char_trn))[:, np.newaxis], axis=0) 
    # this line gives us the results of the logistic regression run on the per character accuracy (spellcheck, essentially) for the training dataset
# logistic_frt_accuracy = np.mean(logistic_letter_mat_frt == np.array(list(char_frt))[:, np.newaxis], axis=0)
    #  this line gives us the results of the logistic regression run on the per character accuracy (spellcheck, essentially) for the test dataset
# lda_trn_accuracy = np.mean(lda_letter_mat_trn == np.array(list(char_trn))[:, np.newaxis], axis=0)
    #calculates the per character accuracy vector using LDA method on a Gaussian distribution for the training dataset
# lda_frt_accuracy = np.mean(lda_letter_mat_frt == np.array(list(char_frt))[:, np.newaxis], axis=0)
    #calculates the per character accuracy vector using LDA method on a Gaussian distribution for the test dataset
# svm_trn_accuracy = np.mean(svm_letter_mat_trn == np.array(list(char_trn))[:, np.newaxis], axis=0)
    #the per character accuracy vector (fraction of characters correctly decoded per attempt) for the training dataset
# svm_frt_accuracy = np.mean(svm_letter_mat_frt == np.array(list(char_frt))[:, np.newaxis], axis=0)
    #the per character accuracy vector (fraction of characters correctly decoded per attempt) for the test dataset

# Step 5: Summary
# Which method performs the best? Why? 
#for this dataset, SVM performs the best. It's accuracy metrics are the highest with a AUC of 0.981 and an accuracy of 0.962.
# SVM also has the fewest false positives and false negatives compared with logistic and LDA

19
dict_keys(['__header__', '__version__', '__globals__', 'Code', 'IndexBegin', 'IndexTag', 'LetterTable', 'Signal', 'Text', 'Type'])
(3420, 400)
(3420, 1)
FRT keys: ['__header__', '__version__', '__globals__', 'BandLow', 'BandUpp', 'Code', 'IndexBegin', 'IndexTag', 'LetterTable', 'Signal', 'Text', 'Type']
raw Signal shape: (1296, 400) raw Type shape: (1296, 1)
processed Signal shape: (1296, 400) processed Type shape: (1296,)
char_frt_size: 27
trn_seq_size: 15 frt_seq_size: 4
first 10 eeg_frt_type labels: [-1 -1  1  1 -1 -1 -1 -1 -1 -1]
first 10 eeg_frt_code rows: [[ 6]
 [ 7]
 [ 8]
 [ 4]
 [ 5]
 [ 9]
 [10]
 [ 3]
 [11]
 [ 2]]
Shapes: (3420, 1) (3420, 1) (3420, 1) (1296, 1) (1296, 1) (1296, 1)
Created model-output arrays with shapes:
logistic_y_trn (3420,) logistic_y_frt (1296,)
lda_y_trn (3420,) lda_y_frt (1296,)
svm_y_trn (3420,) svm_y_frt (1296,)

Logistic (stimulus-level):
  n_events: 3420
  AUC: 0.9766752847029855
  Accuracy @0.5: 0.9532163742690059
  Confusion matrix (rows=true [0,1